In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import random
import os

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import tensorflow as tf

seed = 2023
random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.experimental.numpy.random.seed(seed)
os.environ["TF_CUDNN_DETERMINISTIC"] = "1"
os.environ["TF_DETERMINISTIC_OPS"] = "1"
# Set a fixed value for the hash seed
os.environ["PYTHONHASHSEED"] = str(seed)
import dolfin as df
import time
from utils import *
from utils_training import *
import prepare_data
import seaborn as sns
from mpl_toolkits.axes_grid1 import make_axes_locatable
import pandas as pd

sns.set_theme()
sns.set_context("paper")
sns.set(rc={"xtick.bottom": True, "ytick.left": True})
colors = sns.color_palette("mako").as_hex()
my_cmap = sns.color_palette("viridis", as_cmap=True)

# gpus = tf.config.list_physical_devices('GPU')
# if gpus:
#     for gpu in gpus:
#         tf.config.experimental.set_virtual_device_configuration(gpu,[tf.config.experimental.VirtualDeviceConfiguration(memory_limit=4096)])

In [ ]:
nb_data_origin = 2811
nb_data_used = 2811
save_figs = True

small_data = False  # False=>on utilise toutes les données
if not (small_data):
    nb_data_used = nb_data_origin
level = 2
data = DataLoader(small_data)
agent = Agent(data, small_data)

if not (os.path.exists(f"./images_{nb_data_used}/")) and save_figs:
    os.makedirs(f"./images_{nb_data_used}/")

epochs = [100, 200, 500, 750, 1000, 1250, 1500, 2000]
print(len(epochs))
print(epochs)

indices = list(range(0, len(epochs)))
size_per_fig = 4
nb_rows = 2
size_row = int(len(indices) / nb_rows)

# Erreur minimale d'un modèle sur le jeu de validation

In [ ]:
i = 2000
agent.model.load_weights(f"./models_{nb_data_used}/model_{i}/model_weights")
X_val, Y_val = agent.data.X_val, agent.data.Y_val
Y_pred = agent.model.call(X_val)
domain = X_val[:, :, :, -2]
boundary = X_val[:, :, :, -1]
nb_vert = np.shape(boundary)[1]
domain_prop = tf.reduce_sum(domain, axis=[1, 2]) / nb_vert**2
boundary_prop = tf.reduce_sum(boundary, axis=[1, 2]) / nb_vert**2

diff = (Y_val - Y_pred) * X_val[:, :, :, 1, None]

L2_error = tf.sqrt(
    tf.reduce_mean(
        (diff**2)
        * X_val[:, :, :, -2, None]
        / (domain_prop[:, None, None, None]),
        axis=[1, 2, 3],
    )
    / tf.reduce_mean(
        ((Y_val * X_val[:, :, :, 1, None]) ** 2)
        * X_val[:, :, :, -2, None]
        / (domain_prop[:, None, None, None]),
        axis=[1, 2, 3],
    )
)

argmin = np.argmin(L2_error)
print(argmin)
sorted_min = sorted(range(len(L2_error)), key=lambda k: L2_error[k])
minimal_errors = sorted_min[:1]
print(minimal_errors)


Params = np.load(f"./data_{nb_data_origin}_3_modes/agentParams.npy")[
    0:nb_data_used
]
data_size = np.shape(Params)[0]
nb_val = data_size // 8
nb_train = data_size - nb_val
params_val = Params[nb_train:]


for i in minimal_errors:
    X = data.X_val[i, :, :]
    F = X[:, :, 0] * data.max_norm_F
    U_ref = data.Y_val[i]
    Y = agent.model.call(X[None, :, :, :])

    domains_nan = X[:, :, -2].numpy()
    domains_nan[domains_nan == 0] = np.nan

    plt.figure(figsize=(12, 4))

    ax1 = plt.subplot(131)
    img = ax1.imshow(
        F * domains_nan, cmap=my_cmap, aspect="equal", origin="lower"
    )
    divider = make_axes_locatable(ax1)
    cax = divider.append_axes("bottom", size="5%", pad=0.3)
    ax1.grid(False)
    ax1.set_title(r"$f$", fontsize=20)
    plt.colorbar(img, cax=cax, orientation="horizontal")

    ax2 = plt.subplot(132)
    img = ax2.imshow(
        Y[0, :, :, 0] * X[:, :, 1] * domains_nan,
        cmap=my_cmap,
        aspect="equal",
        origin="lower",
    )
    divider = make_axes_locatable(ax2)
    cax = divider.append_axes("bottom", size="5%", pad=0.3)
    ax2.grid(False)
    ax2.set_title(r"$\phi w_{pred}$", fontsize=20)
    plt.colorbar(img, cax=cax, orientation="horizontal")

    ax3 = plt.subplot(133)
    img = ax3.imshow(
        U_ref[:, :, 0] * X[:, :, 1] * domains_nan,
        cmap=my_cmap,
        aspect="equal",
        origin="lower",
    )
    divider = make_axes_locatable(ax3)
    cax = divider.append_axes("bottom", size="5%", pad=0.3)
    ax3.grid(False)
    ax3.set_title(r"$\phi w_{ref}$", fontsize=20)
    plt.colorbar(img, cax=cax, orientation="horizontal")

    if save_figs:
        plt.savefig(
            f"./images_{nb_data_used}/best_prediction_{L2_error[i]:6f}.png"
        )
    plt.show()

    params = params_val[i]
    print(f"(mu0, mu1, sigma, x_0, y_0, lx, ly, theta) = {params}")
    print(f"{L2_error[i]:6f}")

# Erreur relative en fonction des epochs

## Jeu de validation

In [ ]:
errors = []
Linf_errors = []

for i in epochs:
    agent.model.load_weights(
        f"./models_{nb_data_used}/model_{i}/model_weights"
    )
    X_val, Y_val = agent.data.X_val, agent.data.Y_val
    Y_pred = agent.model.call(X_val)
    domain = X_val[:, :, :, -2]
    boundary = X_val[:, :, :, -1]
    nb_vert = np.shape(boundary)[1]
    domain_prop = tf.reduce_sum(domain, axis=[1, 2]) / nb_vert**2
    boundary_prop = tf.reduce_sum(boundary, axis=[1, 2]) / nb_vert**2

    diff = (Y_val - Y_pred) * X_val[:, :, :, 1, None]

    val = tf.sqrt(
        tf.reduce_mean(
            (diff**2)
            * X_val[:, :, :, -2, None]
            / (domain_prop[:, None, None, None]),
            axis=[1, 2, 3],
        )
        / tf.reduce_mean(
            ((Y_val * X_val[:, :, :, 1, None]) ** 2)
            * X_val[:, :, :, -2, None]
            / (domain_prop[:, None, None, None]),
            axis=[1, 2, 3],
        )
    )

    errors.append(val)
    Linf_errors.append(
        np.max(
            np.max(np.absolute(diff * X_val[:, :, :, -2, None]), axis=1),
            axis=1,
        )
        / np.max(
            np.max(
                np.absolute(
                    Y_val * X_val[:, :, :, 1, None] * X_val[:, :, :, -2, None]
                ),
                axis=1,
            ),
            axis=1,
        )
    )

In [ ]:
j, k = 0, 0
fig, axes = plt.subplots(
    nb_rows,
    size_row,
    figsize=(size_per_fig * size_row, size_per_fig * nb_rows),
)

for i in range(len(indices)):
    if k == size_row:
        k = 0
        j += 1
    sns.histplot(
        data=errors[indices[i]],
        kde=True,
        bins=20,
        color=colors[2],
        edgecolor="k",
        log_scale=True,
        label=str(epochs[indices[i]]),
        stat="proportion",
        legend=True,
        ax=axes[j, k],
    )
    axes[j, k].set_xlabel("$L^2$ relative error")
    axes[j, k].legend()
    axes[j, k].set_xlim(3e-4, 5e-1)
    axes[j, k].set_ylim(0.0, 0.220)
    k += 1

plt.tight_layout()
if save_figs:
    plt.savefig(f"./images_{nb_data_used}/histograms_validation_L2.png")
plt.show()

In [ ]:
j, k = 0, 0
fig, axes = plt.subplots(
    nb_rows,
    size_row,
    figsize=(size_per_fig * size_row, size_per_fig * nb_rows),
)

for i in range(len(indices)):
    if k == size_row:
        k = 0
        j += 1
    sns.histplot(
        data=Linf_errors[indices[i]],
        kde=True,
        bins=20,
        color=colors[2],
        edgecolor="k",
        log_scale=True,
        label=str(epochs[indices[i]]),
        stat="proportion",
        legend=True,
        ax=axes[j, k],
    )
    axes[j, k].set_xlabel("$L^\infty$ relative error")
    axes[j, k].legend()
    axes[j, k].set_xlim(3e-4, 5e-1)
    axes[j, k].set_ylim(0.0, 0.220)
    k += 1

plt.tight_layout()
if save_figs:
    plt.savefig(f"./images_{nb_data_used}/histograms_validation_Linf.png")
plt.show()

In [ ]:
plt.figure(figsize=(10, 5))

"""
On crée des dataframe de la forme
|epoch|error|norm|
où type est L2 ou Linf et epoch est un multiple de 250
"""

size_val = len(errors[0])

epoch_repeat = np.array([])
for epoch in epochs:
    tab = np.repeat(str(epoch), size_val)
    epoch_repeat = np.concatenate([epoch_repeat, tab], axis=0)

tab_L2 = np.repeat(["$L^2$"], size_val * len(epochs))

errors_L2_flatten = np.array(errors).flatten()

df_array_L2 = np.stack([epoch_repeat, errors_L2_flatten, tab_L2], axis=0)

tab_Linf = np.repeat(["$L^\infty$"], size_val * len(epochs))
errors_Linf_flatten = np.array(Linf_errors).flatten()
df_array_Linf = np.stack([epoch_repeat, errors_Linf_flatten, tab_Linf], axis=0)

df_array = np.concatenate([df_array_L2, df_array_Linf], axis=1)

name = ["Epochs", "Error", "Norm"]
df_errors = pd.DataFrame(df_array.transpose(), columns=name)
df_errors = df_errors.astype(
    {"Epochs": "int", "Error": "float64", "Norm": "category"}
)

sns.boxplot(
    data=df_errors, x="Epochs", y="Error", hue="Norm", dodge=True
)  # ,palette="ch:s=.25,rot=-.25"
plt.yscale("log")
plt.ylabel("Relative error")
plt.tight_layout()

if save_figs:
    plt.savefig(f"./images_{nb_data_used}/boxplots_validation_L2_Linf.png")
plt.show()

In [ ]:
plt.figure(figsize=(10, 5))
errors = np.array(errors[:])
print(np.shape(errors))
dataframe = pd.DataFrame(errors.transpose(), columns=epochs)

sns.boxplot(data=dataframe, palette="ch:s=.25,rot=-.25")
plt.xlabel("Training epochs")
plt.ylabel("$L^2$ relative error")
plt.yscale("log")
plt.grid(True, "both", "y")
plt.tight_layout()
plt.savefig(f"./images_{nb_data_used}/boxplots_validation_epochs_L2_error.png")
plt.show()

In [ ]:
epochs_means = list(i * 50 for i in range(1, 41))

print(len(epochs_means))
print(epochs_means)

errors = []
Linf_errors = []

for i in epochs_means:
    agent.model.load_weights(
        f"./models_{nb_data_used}/model_{i}/model_weights"
    )
    X_val, Y_val = agent.data.X_val, agent.data.Y_val
    Y_pred = agent.model.call(X_val)
    domain = X_val[:, :, :, -2]
    boundary = X_val[:, :, :, -1]
    nb_vert = np.shape(boundary)[1]
    domain_prop = tf.reduce_sum(domain, axis=[1, 2]) / nb_vert**2
    boundary_prop = tf.reduce_sum(boundary, axis=[1, 2]) / nb_vert**2

    diff = Y_val * X_val[:, :, :, 1, None] - (Y_pred * X_val[:, :, :, 1, None])

    val = tf.sqrt(
        tf.reduce_mean(
            (diff**2)
            * X_val[:, :, :, -2, None]
            / (domain_prop[:, None, None, None]),
            axis=[1, 2, 3],
        )
        / tf.reduce_mean(
            ((Y_val * X_val[:, :, :, 1, None]) ** 2)
            * X_val[:, :, :, -2, None]
            / (domain_prop[:, None, None, None]),
            axis=[1, 2, 3],
        )
    )

    errors.append(val)
    Linf_errors.append(
        np.max(
            np.max(np.absolute(diff * X_val[:, :, :, -2, None]), axis=1),
            axis=1,
        )
        / np.max(
            np.max(
                np.absolute(
                    Y_val * X_val[:, :, :, 1, None] * X_val[:, :, :, -2, None]
                ),
                axis=1,
            ),
            axis=1,
        )
    )

In [ ]:
means = np.mean(errors, axis=1)
standard_deviation = np.std(errors, axis=1)
maxs = np.max(errors, axis=1)
mins = np.min(errors, axis=1)

plt.figure(figsize=(8, 3))
plt.subplot(1, 2, 1)
plt.semilogy(epochs_means, means, "-+", label="Mean")
plt.xlabel("Epochs")
plt.ylabel("$L^2$ relative error")
plt.semilogy(
    epochs_means, standard_deviation, "-+", label="Standard deviation"
)
plt.xlabel("Epochs")
plt.ylabel("$L^2$ relative error")
plt.grid(True, "both", "both")
plt.legend()

plt.subplot(1, 2, 2)
plt.semilogy(epochs_means, maxs, "-+", label="Maximum")
plt.semilogy(epochs_means, mins, "-+", label="Minimum")
plt.grid(True, "both", "both")
plt.legend()
plt.xlabel("Epochs")
plt.ylabel("$L^2$ relative error")
plt.tight_layout()

if save_figs:
    plt.savefig(f"./images_{nb_data_used}/min_mean_max_L2_error_epochs.png")
plt.show()

# Erreur sur un nouveau jeu de données

In [ ]:
from generate_data import PhiFemSolver

In [ ]:
u_exs, V_exs, dx_exs = [], [], []
L2_error_phi_fem, L2_error_std_fem = [], []
Temps_phi, Temps_std = [], []

F, Phi, params = create_FG_numpy(300, 64)
solver = PhiFemSolver(nb_cell=64 - 1, params=params, phi_vector=Phi)
W_phi_fem, Phi_64 = solver.solve_several()
print(f"{np.shape(W_phi_fem)=}")

errors_fno, times_fno = [], []

for j in epochs:
    print(f"Epoch : {j}")
    agent.model.load_weights(
        f"./models_{nb_data_used}/model_{j}/model_weights"
    )
    L2_error_fno = []
    Temps_fno = []
    X = generate_manual_new_data_numpy(Phi_64, F / data.max_norm_F)
    Y_pred = agent.model.call(X)

    domain = X[:, :, :, -2]
    boundary = X[:, :, :, -1]
    nb_vert = np.shape(boundary)[1]
    domain_prop = tf.reduce_sum(domain, axis=[1, 2]) / nb_vert**2
    boundary_prop = tf.reduce_sum(boundary, axis=[1, 2]) / nb_vert**2
    diff = W_phi_fem[:, :, :] * X[:, :, :, 1] - (
        Y_pred[:, :, :, 0] * X[:, :, :, 1]
    )

    L2_error_fno = tf.sqrt(
        tf.reduce_mean(
            (diff**2) * X[:, :, :, -2] / (domain_prop[:, None, None]),
            axis=[1, 2],
        )
        / tf.reduce_mean(
            ((W_phi_fem * X[:, :, :, 1]) ** 2)
            * X[:, :, :, -2]
            / (domain_prop[:, None, None]),
            axis=[1, 2],
        )
    )

    errors_fno.append(L2_error_fno)

In [ ]:
j, k = 0, 0
fig, axes = plt.subplots(
    nb_rows,
    size_row,
    figsize=(size_per_fig * size_row, size_per_fig * nb_rows),
)

for i in range(len(indices)):
    if k == size_row:
        k = 0
        j += 1
    sns.histplot(
        data=errors_fno[indices[i]],
        kde=True,
        bins=20,
        color=colors[2],
        edgecolor="k",
        log_scale=True,
        label=str(epochs[indices[i]]),
        stat="proportion",
        legend=True,
        ax=axes[j, k],
    )
    axes[j, k].set_xlabel("$L^2$ relative error")
    axes[j, k].legend()
    axes[j, k].set_xlim(5e-4, 2e0)
    axes[j, k].set_ylim(0.0, 0.24)
    k += 1

plt.tight_layout()
if save_figs:
    plt.savefig(f"./images_{nb_data_used}/histograms_new_data_L2.png")
plt.show()

In [ ]:
import pandas as pd

plt.figure(figsize=(10, 5))

"""
On crée des dataframe de la forme
|epoch|error|norm|
où type est L2 ou Linf et epoch est un multiple de 250
"""

size_val = len(errors_fno[0])

epoch_repeat = np.array([])
for epoch in epochs:
    tab = np.repeat(str(epoch), size_val)
    epoch_repeat = np.concatenate([epoch_repeat, tab], axis=0)

tab_L2 = np.repeat(["$L^2$"], size_val * len(epochs))

errors_L2_flatten = np.array(errors_fno).flatten()

df_array_L2 = np.stack([epoch_repeat, errors_L2_flatten, tab_L2], axis=0)

df_array = np.concatenate([df_array_L2], axis=1)

name = ["Epochs", "Error", "Norm"]
df_errors = pd.DataFrame(df_array.transpose(), columns=name)
df_errors = df_errors.astype(
    {"Epochs": "int", "Error": "float64", "Norm": "category"}
)

sns.boxplot(
    data=df_errors, x="Epochs", y="Error", hue="Norm", dodge=True
)  # ,palette="ch:s=.25,rot=-.25"
plt.yscale("log")
plt.ylabel("Relative error")
plt.tight_layout()

save_figs = True
if save_figs:
    plt.savefig(f"./images_{nb_data_used}/boxplots_new_data_L2.png")
plt.show()

In [ ]:
errors_fno = []
epochs_means = list(i * 50 for i in range(1, 41))

for j in epochs_means:
    print(f"Epoch : {j}")
    agent.model.load_weights(
        f"./models_{nb_data_used}/model_{j}/model_weights"
    )
    L2_error_fno = []
    Temps_fno = []
    X = generate_manual_new_data_numpy(Phi_64, F / data.max_norm_F)
    Y_pred = agent.model.call(X)

    domain = X[:, :, :, -2]
    boundary = X[:, :, :, -1]
    nb_vert = np.shape(boundary)[1]
    domain_prop = tf.reduce_sum(domain, axis=[1, 2]) / nb_vert**2
    boundary_prop = tf.reduce_sum(boundary, axis=[1, 2]) / nb_vert**2
    diff = W_phi_fem[:, :, :] * X[:, :, :, 1] - (
        Y_pred[:, :, :, 0] * X[:, :, :, 1]
    )

    L2_error_fno = tf.sqrt(
        tf.reduce_mean(
            (diff**2) * X[:, :, :, -2] / (domain_prop[:, None, None]),
            axis=[1, 2],
        )
        / tf.reduce_mean(
            ((W_phi_fem * X[:, :, :, 1]) ** 2)
            * X[:, :, :, -2]
            / (domain_prop[:, None, None]),
            axis=[1, 2],
        )
    )

    errors_fno.append(L2_error_fno)

In [ ]:
means = np.mean(errors_fno, axis=1)
standard_deviation = np.std(errors_fno, axis=1)
maxs = np.max(errors_fno, axis=1)
mins = np.min(errors_fno, axis=1)

plt.figure(figsize=(8, 4))
plt.subplot(1, 2, 1)
plt.semilogy(epochs_means, means, "-+", label="Mean")
plt.xlabel("Epochs")
plt.ylabel("$L^2$ relative error")
plt.semilogy(
    epochs_means, standard_deviation, "-+", label="Standard deviation"
)
plt.xlabel("Epochs")
plt.ylabel("$L^2$ relative error")
plt.grid(True, "both", "both")
plt.legend()

plt.subplot(1, 2, 2)
plt.semilogy(epochs_means, maxs, "-+", label="Maximum")
plt.semilogy(epochs_means, mins, "-+", label="Minimum")
plt.grid(True, "both", "both")
plt.legend()
plt.xlabel("Epochs")
plt.ylabel("$L^2$ relative error")
plt.tight_layout()

if save_figs:
    plt.savefig(
        f"./images_{nb_data_used}/min_mean_max_L2_error_epochs_new_data.png"
    )
plt.show()